In [6]:
import pandas as pd


pd.set_option("display.max_columns", None)
pd.set_option("display.width", 160)

df = pd.read_csv("crypto_raw.csv")
display(df)


,Category,Symbol,Name,Price,Change,Change %,Market Cap,Volume,Volume InCurrency (24hr),Total Volume AllCurrencies (24hr),CirculatingSupply,52 WkChange %
0,Most Active,UUSDT-USD,Tether USDt USD,1.00-0.00(-0.01%),-0.00,-0.01%,185.668B,58.329B,58.329B,58.329B,185.632B,-0.07%
1,Most Active,MMPRO31258-USD,Max Property USD,0.01-0.00(-0.01%),-0.00,-0.01%,0,45.2B,45.2B,45.2B,0,-99.96%
2,Most Active,BBTC-USD,Bitcoin USD,"89,458.34+5.80(+0.01%)",5.80,+0.01%,1.785T,41.591B,41.591B,41.591B,19.958M,-10.54%
3,Top Gainers,HHOCAI-USD,Heroes of Crypto AI USD,"0.01+0.01(+7,889.33%)",0.01,"+7,889.33%",0,"37,220","37,220","37,220",0,+364.26%
4,Top Gainers,FFELIS-USD,Felis USD,"0.00+0.00(+2,936.48%)",0.00,"+2,936.48%",0,"117,294","117,294","117,294",0,-66.67%
5,Top Gainers,DDOGO29570-USD,DOGO USD,"0.00+0.00(+2,919.59%)",0.00,"+2,919.59%",0,"83,141","83,141","83,141",0,+500.00%
6,Top Losers,BBPX10720-USD,Black Phoenix USD,0.02-2.80(-99.32%),-2.80,-99.32%,0,1.411M,1.411M,1.411M,0,-97.41%
7,Top Losers,CCAT32486-USD,NOT USD,0.00-0.17(-98.05%),-0.17,-98.05%,0,"351,128","351,128","351,128",0,"+180,576.77%"
8,Top Losers,TTRUMP31874-USD,PEPE USD,0.00-0.00(-97.39%),-0.00,-97.39%,0,"324,925","324,925","324,925",0,-97.10%


In [9]:
import re

def extract_first_number(text: str):
    #Retourne le premier nombre rencontré dans une chaîne, en float
    if not isinstance(text, str):
        return None
    match = re.search(r"[-+]?\d[\d,]*\.?\d*", text)
    if not match:
        return None
    return float(match.group().replace(",", ""))


def to_float(x):
    if isinstance(x, (int, float)):
        return float(x)

    if not isinstance(x, str):
        return None

    x = x.replace(",", "").strip()
    if x == "":
        return None
    try:
        return float(x)
    except ValueError:
        return None



def percent_to_float(text: str):
    if not isinstance(text, str):
        return None
    text = text.replace("%", "").replace(",", "").strip()
    if text == "":
        return None
    try:
        return float(text)
    except ValueError:
        return None


def parse_with_suffix(text: str):
    if not isinstance(text, str):
        return None

    text = text.replace(",", "").strip()
    if text == "":
        return None

    suffix = text[-1] if text[-1] in "KMBT" else ""
    number_part = text[:-1] if suffix else text

    try:
        value = float(number_part)
    except ValueError:
        return None

    factor = {
        "": 1,
        "K": 1e3,
        "M": 1e6,
        "B": 1e9,
        "T": 1e12,
    }.get(suffix, 1)

    return value * factor


In [10]:
df_clean = df.copy()

# 1) Nettoyer Price 
if "Price" in df_clean.columns:
    df_clean["Price"] = df_clean["Price"].apply(extract_first_number)

# 2) Nettoyer Change et Change %
if "Change" in df_clean.columns:
    df_clean["Change"] = df_clean["Change"].apply(to_float)

if "Change %" in df_clean.columns:
    df_clean["Change %"] = df_clean["Change %"].apply(percent_to_float)

# 3) Colonnes avec K / M / B / T
cols_with_suffix = [
    "Market Cap",
    "Volume",
    "Volume InCurrency (24hr)",
    "Total Volume AllCurrencies (24hr)",
    "CirculatingSupply",
]

for col in cols_with_suffix:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].apply(parse_with_suffix)

# 4) 52 WkChange % 
if "52 WkChange %" in df_clean.columns:
    df_clean["52 WkChange %"] = df_clean["52 WkChange %"].apply(percent_to_float)

print(" Données nettoyées ")
display(df_clean)

df_clean.to_csv("crypto_clean.csv", index=False)



 Données nettoyées 


,Category,Symbol,Name,Price,Change,Change %,Market Cap,Volume,Volume InCurrency (24hr),Total Volume AllCurrencies (24hr),CirculatingSupply,52 WkChange %
0,Most Active,UUSDT-USD,Tether USDt USD,1.00,-0.00,-0.01,1.856680e+11,5.832900e+10,5.832900e+10,5.832900e+10,1.856320e+11,-0.07
1,Most Active,MMPRO31258-USD,Max Property USD,0.01,-0.00,-0.01,0.000000e+00,4.520000e+10,4.520000e+10,4.520000e+10,0.000000e+00,-99.96
2,Most Active,BBTC-USD,Bitcoin USD,89458.34,5.80,0.01,1.785000e+12,4.159100e+10,4.159100e+10,4.159100e+10,1.995800e+07,-10.54
3,Top Gainers,HHOCAI-USD,Heroes of Crypto AI USD,0.01,0.01,7889.33,0.000000e+00,3.722000e+04,3.722000e+04,3.722000e+04,0.000000e+00,364.26
4,Top Gainers,FFELIS-USD,Felis USD,0.00,0.00,2936.48,0.000000e+00,1.172940e+05,1.172940e+05,1.172940e+05,0.000000e+00,-66.67
5,Top Gainers,DDOGO29570-USD,DOGO USD,0.00,0.00,2919.59,0.000000e+00,8.314100e+04,8.314100e+04,8.314100e+04,0.000000e+00,500.00
6,Top Losers,BBPX10720-USD,Black Phoenix USD,0.02,-2.80,-99.32,0.000000e+00,1.411000e+06,1.411000e+06,1.411000e+06,0.000000e+00,-97.41
7,Top Losers,CCAT32486-USD,NOT USD,0.00,-0.17,-98.05,0.000000e+00,3.511280e+05,3.511280e+05,3.511280e+05,0.000000e+00,180576.77
8,Top Losers,TTRUMP31874-USD,PEPE USD,0.00,-0.00,-97.39,0.000000e+00,3.249250e+05,3.249250e+05,3.249250e+05,0.000000e+00,-97.10
